Inatalação das bibliotecao

In [ ]:
!pip install deepface
!pip install keras
!pip install opencv-python-headless tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=6355242775694501301c0e72016fcfd078740aceec76b744f2f2e40b6cea8962
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


Variáveis de entreda e saída

In [ ]:
input_video_path="/content/drive/MyDrive/Colab Notebooks/Unlocking Facial Recognition_ Diverse Activities Analysis.mp4"
output_video_path_emotions="/content/output_video_emotions.mp4"
output_video_path_pose="/content/output_video_pose.mp4"
output_text_path="/content/output_text.txt"
output_audio_path="/content/output_audio.wav"
output_text_path_sentences="/content/output_text_sentences.txt"
output_text_path_punctuation="/content/output_text_punctuation.txt"
output_text_path_summarization="/content/output_text_summarization.txt"
output_text_path_video_emotions="/content/output_text_path_video_emotions.txt"
output_text_path_video_emotions_summarization="/content/output_text_path_video_emotions_summarization.txt"

Detectção de emoção

In [ ]:
import cv2
from deepface import DeepFace
import os
import numpy as np
from tqdm import tqdm

def detect_emotions(video_path, output_path):
  cap = cv2.VideoCapture(video_path)

  if not cap.isOpened():
    print("Error opening video file")
    return

  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = cap.get(cv2.CAP_PROP_FPS)
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

  dominant_emotions = []
  for _ in tqdm(range(total_frames), desc="Processing frames"):
    ret, frame = cap.read()
    if not ret:
      break

    results = DeepFace.analyze(
            frame,
            actions=['emotion'],
            enforce_detection=False,
            detector_backend='opencv',
            align=False
        )

        # Caso retorne um único dicionário, converte para lista
    if isinstance(results, dict):
        results = [results]


    for face in results:
      x = face['region']['x']
      y = face['region']['y']
      w = face['region']['w']
      h = face['region']['h']

      dominant_emotion = face['dominant_emotion']
      dominant_emotions.append({
          "frame": _,
          "emotion": dominant_emotion
      })


      cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

      cv2.putText(frame, dominant_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    out.write(frame)

  cap.release()
  out.release()
  cv2.destroyAllWindows()

  with open(output_text_path_video_emotions, "w", encoding="utf-8") as f:
    for item in dominant_emotions:
        f.write(f"At frame {item['frame']}, a person expressed {item['emotion']}.")


detect_emotions(input_video_path, output_video_path_emotions)

25-06-01 23:17:49 - Directory /root/.deepface has been created
25-06-01 23:17:49 - Directory /root/.deepface/weights has been created


Processing frames:   0%|          | 0/3326 [00:00<?, ?it/s]

25-06-01 23:17:53 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5

100%|██████████| 5.98M/5.98M [00:00<00:00, 309MB/s]
Processing frames: 100%|██████████| 3326/3326 [06:21<00:00,  8.73it/s]


Detecçaão de Pose

In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
grpcio

In [ ]:
import mediapipe as mp
import cv2
from tqdm import tqdm
from google.colab.patches import cv2_imshow

def detect_pose(video_path, output_path):
  mp_drawing = mp.solutions.drawing_utils
  mp_pose = mp.solutions.pose
  pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

  cap = cv2.VideoCapture(video_path)
  if not cap.isOpened():
    print("Error opening video file")
    return

  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = cap.get(cv2.CAP_PROP_FPS)
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

  for _ in tqdm(range(total_frames), desc="Processing frames"):
    ret, frame = cap.read()
    if not ret:
      break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = pose.process(frame_rgb)

    if results.pose_landmarks:
      mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    out.write(frame)


  cap.release()
  out.release()
  cv2.destroyAllWindows()

detect_pose(input_video_path, output_video_path_pose)

Processing frames: 100%|██████████| 3326/3326 [01:57<00:00, 28.22it/s]


Transcrição de áudio

In [ ]:
!pip install moviepy speechrecognition pydub
!pip install deepmultilingualpunctuation


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
import  moviepy.editor as mp
import speech_recognition as sr
from pydub import AudioSegment
import os
from tqdm import tqdm
from deepmultilingualpunctuation import PunctuationModel

def extract_audio_form_video(video_path, output_audio_path):
  clip = mp.VideoFileClip(video_path)
  clip.audio.write_audiofile(output_audio_path)

def transcribe_audio(audio_path, output_text_path):
  recognizer = sr.Recognizer()
  with sr.AudioFile(audio_path) as source:
    audio_data = recognizer.record(source)
    text = recognizer.recognize_google(audio_data, language="en-US")
    with open(output_text_path, "w", encoding="utf-8") as text_file:
      text_file.write(text)

def insert_text_punctuation(input_text_path, output_text_path_punctuation):

  with open(input_text_path, "r", encoding="utf-8") as f:
    text = f.read()

  model = PunctuationModel()
  text_pontuado = model.restore_punctuation(text)

  with open(output_text_path_punctuation, "w", encoding="utf-8") as f:
    f.write(text_pontuado)

extract_audio_form_video(input_video_path, output_audio_path)
transcribe_audio(output_audio_path, output_text_path)
insert_text_punctuation(output_text_path, output_text_path_punctuation)

MoviePy - Writing audio in /content/output_audio.wav


MoviePy - Done.


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


Categorização

In [ ]:
!pip install scikit-learn gensim nltk scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 60.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn import metrics

def classify_sentences (input_text, output_text_sentences):
  text = []
  labels = [
      "technology",
      "technology",
      "technology",
      "reading",
      "conversation",
      "movement",
      "emotion",
      "emotion",
      "details",
      "diversity",
      "technology",
      "technology"
  ]

  with open(input_text, "r", encoding="utf-8") as f:
    raw_text = f.read()

  # Quebra em frases
  text = [s.strip() for s in raw_text.split('.') if s.strip()]

  print(len(text))
  x_train, x_test, y_train, y_test = train_test_split(text[:9], labels[:9], test_size=0.2, random_state=42)

  model = make_pipeline(TfidfVectorizer(), MultinomialNB())
  model.fit(x_train, y_train)

  predicted = model.predict(x_test)
  with open(output_text_sentences, "w", encoding="utf-8") as f:
    f.write("Metrics:\n")
    f.write(metrics.classification_report(y_test, predicted, zero_division=0))

    # Acurácia
    accuracy = metrics.accuracy_score(y_test, predicted)
    f.write(f"Acurácia: {accuracy:.2f}")

    test_sentences = [
        "a man focuses intently on a tablet, his brow furrowed in concentration",
        "two friends laugh loudly while watching a funny video together",
        "a woman flips through a book, absorbed by its content",
        "a child waves enthusiastically at the camera, smiling widely",
        "people move through the space naturally, creating a constant shift in the environment",
        "a man and woman have a heated discussion, their gestures growing more animated",
        "a face scanner tracks multiple individuals entering a busy lobby",
        "subtle facial expressions reveal a range of unspoken thoughts",
        "visual markers trace the contours of each unique face in the crowd",
        "the diversity of ages and appearances highlights human uniqueness"
    ]

    f.write("\n\nNew predictions:\n")
    predictions = model.predict(test_sentences)
    for sentence, label in zip(test_sentences, predictions):
        f.write(f"{sentence} ---> {label}\n")

classify_sentences(output_text_path_punctuation, output_text_path_sentences)


9


Sumarização


In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", device=-1)

def summarize_text(input_text_path, max_length, output_text_path_sum):
  with open(input_text_path, "r", encoding="utf-8") as f:
    text = f.read()

  #chunks = chunk_text(text)

  #summaries = []
  #for chunk in chunks:
  #    summary = summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]
  #    summaries.append(summary)

  # Unir todos os resumos
  #final_summary = "\n".join(summaries)
  summary = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]

  with open(output_text_path_sum, "w", encoding="utf-8") as f:
    f.write(summary)

def chunk_text(text, max_chars=1500):
    sentences = text.split('.')
    chunks = []
    current = ""

    for sentence in sentences:
        if len(current) + len(sentence) < max_chars:
            current += sentence + "."
        else:
            chunks.append(current.strip())
            current = sentence + "."
    if current:
        chunks.append(current.strip())
    return chunks

#summarize_text(output_text_path_punctuation, 200, output_text_path_summarization)
summarize_text(output_text_path_video_emotions, 80, output_text_path_video_emotions_summarization)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (33286 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self